# Ruble Volatility Project — Part 1: Data Input & Cleaning

This notebook is the first step in a broader project focused on modeling and analyzing the volatility of the Russian ruble (RUB) from 1992 to 2025.  
Here, I load, clean, and consolidate exchange rate data, along with key macroeconomic indicators (GDP, FDI, oil prices, etc.), to prepare for modeling and exploratory analysis in future notebooks.

In this notebook (`01_Data_Input_and_Cleaning.ipynb`), I will:

1. Load historical RUB exchange rate data (1992–2025)
2. Clean and merge missing August 2025 data
3. Integrate key macroeconomic variables (GDP, FDI, inflation, Oil Prices)

This notebook focuses on **loading and preparing the data**.  
Exploratory data analysis (visualizations, comparisons, and statistical summaries) will be done in `02_Exploratory_Data_Analysis.ipynb`.

In [216]:
import pandas as pd
import numpy as np

### I. Loading Raw Currency Data

Loading historical RUB exchange rates from 1992 to 2025.

In [217]:
# data source: https://www.kaggle.com/datasets/fedorkurushin/rub-usd-historical-data
data = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/RC_F01_07_1992_T04_02_2025.csv", parse_dates=["date"])
data.columns = ["currency_rate", "date"]
data

,currency_rate,date
0,125.2600,1992-07-01
1,134.8000,1992-07-03
2,130.5000,1992-07-08
3,130.3000,1992-07-10
4,130.2000,1992-07-15
...,...,...
7514,97.9658,2025-01-29
7515,98.0126,2025-01-30
7516,98.0062,2025-01-31
7517,97.8107,2025-02-01


Loading the most recent exchange rate data for August 2025, which was missing from the previous dataset.

In [218]:
# data source: https://tradingeconomics.com/russia/currency
data_25 = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/Rtsudcur.csv")
data_25

,#Date;Value 18:50 MSK
0,2025-08-22;80.7498
1,2025-08-21;80.2548
2,2025-08-20;80.1045
3,2025-08-19;80.3466
4,2025-08-18;80.4256
...,...
160,2025-01-10;101.9146
161,2025-01-09;102.2911
162,2025-01-08;101.6797
163,2025-01-06;101.6797


### II. Data Cleaning and Preprocessing

#### 1. Fixing Column Names and Formats

In [219]:
# convert date to datetime
data["date"] = pd.to_datetime(data["date"])

In [220]:
# rows where currency_rate is NA for data
print(data[data["currency_rate"].isna()])

Empty DataFrame
Columns: [currency_rate, date]
Index: []


In [221]:
# split date and rate into separate columns for data_25
data_25[["date", "currency_rate_25"]] = data_25["#Date;Value 18:50 MSK"].str.split(";", expand=True)
# convert to datetime and numeric
data_25["date"] = pd.to_datetime(data_25["date"])
data_25["currency_rate_25"] = pd.to_numeric(data_25["currency_rate_25"])
# drop original mixed column
data_25 = data_25.drop(columns=["#Date;Value 18:50 MSK"])
# sort by date_25 ascending
data_25 = data_25.sort_values("date", ascending=True).reset_index(drop=True)
data_25

,date,currency_rate_25
0,2025-01-03,101.6797
1,2025-01-06,101.6797
2,2025-01-08,101.6797
3,2025-01-09,102.2911
4,2025-01-10,101.9146
...,...,...
160,2025-08-18,80.4256
161,2025-08-19,80.3466
162,2025-08-20,80.1045
163,2025-08-21,80.2548


In [222]:
# rows where currency_rate is NA for data_25
print(data_25[data_25["currency_rate_25"].isna()])

Empty DataFrame
Columns: [date, currency_rate_25]
Index: []


In [223]:
# check date columns types
print(data["date"].dtype)
print(data_25["date"].dtype)

datetime64[ns]
datetime64[ns]


#### 2. Merging Data

In [224]:
merged_data = pd.merge(data, data_25, on="date", how="outer")
merged_data

,currency_rate,date,currency_rate_25
0,125.26,1992-07-01,NaN
1,134.80,1992-07-03,NaN
2,130.50,1992-07-08,NaN
3,130.30,1992-07-10,NaN
4,130.20,1992-07-15,NaN
...,...,...,...
7665,NaN,2025-08-18,80.4256
7666,NaN,2025-08-19,80.3466
7667,NaN,2025-08-20,80.1045
7668,NaN,2025-08-21,80.2548


In [225]:
merged_data["date"] = pd.to_datetime(merged_data["date"], errors="coerce")
merged_data["date"] = merged_data["date"].dt.strftime("%Y-%m-%d")
merged_data

,currency_rate,date,currency_rate_25
0,125.26,1992-07-01,NaN
1,134.80,1992-07-03,NaN
2,130.50,1992-07-08,NaN
3,130.30,1992-07-10,NaN
4,130.20,1992-07-15,NaN
...,...,...,...
7665,NaN,2025-08-18,80.4256
7666,NaN,2025-08-19,80.3466
7667,NaN,2025-08-20,80.1045
7668,NaN,2025-08-21,80.2548


In [226]:
# fill currency_rate with currency_rate_25 where currency_rate is NA
merged_data["currency_rate"] = merged_data["currency_rate"].combine_first(merged_data["currency_rate_25"])
merged_data

,currency_rate,date,currency_rate_25
0,125.2600,1992-07-01,NaN
1,134.8000,1992-07-03,NaN
2,130.5000,1992-07-08,NaN
3,130.3000,1992-07-10,NaN
4,130.2000,1992-07-15,NaN
...,...,...,...
7665,80.4256,2025-08-18,80.4256
7666,80.3466,2025-08-19,80.3466
7667,80.1045,2025-08-20,80.1045
7668,80.2548,2025-08-21,80.2548


In [227]:
merged_data = merged_data.drop(columns=["currency_rate_25"])
data=merged_data
data

,currency_rate,date
0,125.2600,1992-07-01
1,134.8000,1992-07-03
2,130.5000,1992-07-08
3,130.3000,1992-07-10
4,130.2000,1992-07-15
...,...,...
7665,80.4256,2025-08-18
7666,80.3466,2025-08-19
7667,80.1045,2025-08-20
7668,80.2548,2025-08-21


In [228]:
data["year"] = pd.to_datetime(merged_data["date"]).dt.year
data["month"] = pd.to_datetime(merged_data["date"]).dt.month
data.tail()

,currency_rate,date,year,month
7665,80.4256,2025-08-18,2025,8
7666,80.3466,2025-08-19,2025,8
7667,80.1045,2025-08-20,2025,8
7668,80.2548,2025-08-21,2025,8
7669,80.7498,2025-08-22,2025,8


#### 3. Loading Macroeconomic Data

To model and understand ruble volatility from 1992 to 2025, I incorporate several macroeconomic variables that are known to influence exchange rates in emerging markets.

* 		GDP_per_capita (current US $)

	Captures price level changes across the economy — a broader measure of inflation. Useful for understanding real versus nominal economic growth and how inflationary pressures affect currency valuation

* 		GDP_deflator (annual %)

	Captures price level changes across the economy — a broader measure of inflation. Useful for understanding real versus nominal economic growth and how inflationary pressures affect currency valuation.

* 		FD_investment (Foreign Direct Investment)

	Measures long-term capital inflows. Rising FDI generally strengthens a currency, as it reflects investor confidence. Capital flight or reduced FDI often weakens the ruble.

* 		Inflation (CPI or general inflation)

	High inflation tends to erode purchasing power, leading to depreciation of the ruble. Low and stable inflation helps maintain exchange rate stability.

* 		Brent_Crude_Oil (USD per barrel)

	Oil exports are a major revenue source for Russia. The ruble is highly sensitive to oil prices — when oil prices rise, the ruble typically appreciates; when they fall, the ruble weakens.

* ❗❗ Metrics_2025 (Manually collected 2025 estimates) ❗❗

Because 2025 is not yet complete, I manually collected provisional data to extend the time series. While these values may later be revised, they allow me to build forecasts and test models using the most up-to-date available estimates.


In [229]:
GDP_per_capita = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/GDP per capita (current US$).csv",skiprows=4)
GDP_deflator = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/GDP deflator (annual %).csv", skiprows=4)
FD_investment = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/Foreign direct investment.csv", skiprows=4)
Inflation = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/Inflation.csv", skiprows=4)
Brent_Crude_Oil = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/chart_20250912T211212.csv")
Metrics_2025 = pd.read_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/2025 Metrics.csv")

In [230]:
GDP_per_capita_RUS = GDP_per_capita[GDP_per_capita["Country Code"] == "RUS"]

GDP_per_capita_RUS = GDP_per_capita_RUS.drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"]).T
GDP_per_capita_RUS = GDP_per_capita_RUS.reset_index()
GDP_per_capita_RUS.columns = ["year", "GDP_per_capita"]
GDP_per_capita_RUS = GDP_per_capita_RUS[GDP_per_capita_RUS["year"].str.isdigit()]
GDP_per_capita_RUS = GDP_per_capita_RUS.dropna()


if 2025 not in GDP_per_capita_RUS["year"].astype(int).values:
    row_2025 = Metrics_2025.loc[Metrics_2025["Metric"] == "GDP_per_capita", ["year", "Value"]].rename(columns={"Value": "GDP_per_capita"})
    GDP_per_capita_RUS = pd.concat([GDP_per_capita_RUS, row_2025], ignore_index=True)
    GDP_per_capita_RUS["year"] = GDP_per_capita_RUS["year"].astype(int)
    GDP_per_capita_RUS = GDP_per_capita_RUS.sort_values("year").reset_index(drop=True)

GDP_per_capita_RUS.tail()

,year,GDP_per_capita
33,2021,12425.029297
34,2022,15619.614258
35,2023,14159.387695
36,2024,14889.018555
37,2025,14260.000000


In [231]:
GDP_deflator_RUS = GDP_deflator[GDP_deflator["Country Code"] == "RUS"]
GDP_deflator_RUS = GDP_deflator_RUS.drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"]).T
GDP_deflator_RUS = GDP_deflator_RUS.reset_index()
GDP_deflator_RUS.columns = ["year", "GDP_deflator"]
GDP_deflator_RUS = GDP_deflator_RUS[GDP_deflator_RUS["year"].str.isdigit()]
GDP_deflator_RUS["year"] = GDP_deflator_RUS["year"].astype(int)


if 2025 not in GDP_deflator_RUS["year"].values:
    row_2025 = Metrics_2025.loc[Metrics_2025["Metric"] == "GDP_deflator", ["year", "Value"]].rename(columns={"Value": "GDP_deflator"})
    row_2025["year"] = row_2025["year"].astype(int)
    GDP_deflator_RUS = pd.concat([GDP_deflator_RUS, row_2025], ignore_index=True)
    GDP_deflator_RUS = GDP_deflator_RUS.sort_values("year").reset_index(drop=True)

GDP_deflator_RUS.tail()

,year,GDP_deflator
61,2021,18.209101
62,2022,18.184689
63,2023,7.998571
64,2024,9.275671
65,2025,8.200000


In [232]:
FD_investment_RUS = FD_investment[FD_investment["Country Code"] == "RUS"]
FD_investment_RUS = FD_investment_RUS.drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"]).T
FD_investment_RUS = FD_investment_RUS.reset_index()
FD_investment_RUS.columns = ["year", "FD_investment"]
FD_investment_RUS = FD_investment_RUS[FD_investment_RUS["year"].str.isdigit()]
FD_investment_RUS = FD_investment_RUS.dropna()
FD_investment_RUS["year"] = FD_investment_RUS["year"].astype(int)

if 2025 not in FD_investment_RUS["year"].values:
    row_2025 = Metrics_2025.loc[Metrics_2025["Metric"] == "FD_investment", ["year", "Value"]].rename(columns={"Value": "FD_investment"})
    row_2025["year"] = row_2025["year"].astype(int)
    FD_investment_RUS = pd.concat([FD_investment_RUS, row_2025], ignore_index=True)
    FD_investment_RUS = FD_investment_RUS.sort_values("year").reset_index(drop=True)

FD_investment_RUS.tail()

,year,FD_investment
29,2021,4.045000e+10
30,2022,-3.980094e+10
31,2023,-1.004511e+10
32,2024,-8.175980e+09
33,2025,3.300000e+09


In [233]:
Inflation_RUS = Inflation[Inflation["Country Code"] == "RUS"]
Inflation_RUS = Inflation_RUS.drop(columns=["Country Name", "Country Code", "Indicator Name", "Indicator Code"]).T
Inflation_RUS = Inflation_RUS.reset_index()
Inflation_RUS.columns = ["year", "Inflation"]
Inflation_RUS = Inflation_RUS[Inflation_RUS["year"].str.isdigit()]
Inflation_RUS["year"] = Inflation_RUS["year"].astype(int)
Inflation_RUS = Inflation_RUS.dropna()

if 2025 not in Inflation_RUS["year"].values:
    row_2025 = Metrics_2025.loc[Metrics_2025["Metric"] == "Inflation", ["year", "Value"]].rename(columns={"Value": "Inflation"})
    row_2025["year"] = row_2025["year"].astype(int)
    Inflation_RUS = pd.concat([Inflation_RUS, row_2025], ignore_index=True)
    Inflation_RUS = Inflation_RUS.sort_values("year").reset_index(drop=True)

Inflation_RUS.tail()

,year,Inflation
31,2021,18.209101
32,2022,18.184689
33,2023,7.998571
34,2024,9.275671
35,2025,8.140000


In [234]:
Brent_Crude_Oil["year"] = pd.to_datetime(Brent_Crude_Oil["Date"]).dt.year
Brent_Crude_Oil["month"] = pd.to_datetime(Brent_Crude_Oil["Date"]).dt.month
Brent_Crude_Oil = Brent_Crude_Oil.rename(columns={'Value': 'Brent_price'})
Brent_Crude_Oil = Brent_Crude_Oil[Brent_Crude_Oil["year"]>1991]

Brent_Crude_Oil.tail()

,Date,Brent_price,year,month
456,05/30/2025,64.32,2025,5
457,06/30/2025,68.15,2025,6
458,07/31/2025,73.43,2025,7
459,08/29/2025,67.83,2025,8
460,09/08/2025,65.44,2025,9


In [235]:
GDP_per_capita_RUS["year"] = GDP_per_capita_RUS["year"].astype(int)

data = pd.merge(data, GDP_per_capita_RUS, on="year", how="left")
data.tail()

,currency_rate,date,year,month,GDP_per_capita
7665,80.4256,2025-08-18,2025,8,14260.0
7666,80.3466,2025-08-19,2025,8,14260.0
7667,80.1045,2025-08-20,2025,8,14260.0
7668,80.2548,2025-08-21,2025,8,14260.0
7669,80.7498,2025-08-22,2025,8,14260.0


In [236]:
Brent_Crude_Oil_merge = Brent_Crude_Oil[['year', 'month', 'Brent_price']].copy() 
data = pd.merge(data, Brent_Crude_Oil_merge, on=['year', 'month'], how='left')

In [237]:
GDP_deflator_RUS["year"] = GDP_deflator_RUS["year"].astype(int)
FD_investment_RUS["year"] = FD_investment_RUS["year"].astype(int)
Inflation_RUS["year"] = Inflation_RUS["year"].astype(int)
data = pd.merge(data, GDP_deflator_RUS, on="year", how="left")
data = pd.merge(data, FD_investment_RUS, on="year", how="left")
data = pd.merge(data, Inflation_RUS, on="year", how="left")

In [238]:
data.tail()

,currency_rate,date,year,month,GDP_per_capita,Brent_price,GDP_deflator,FD_investment,Inflation
7665,80.4256,2025-08-18,2025,8,14260.0,67.83,8.2,3.300000e+09,8.14
7666,80.3466,2025-08-19,2025,8,14260.0,67.83,8.2,3.300000e+09,8.14
7667,80.1045,2025-08-20,2025,8,14260.0,67.83,8.2,3.300000e+09,8.14
7668,80.2548,2025-08-21,2025,8,14260.0,67.83,8.2,3.300000e+09,8.14
7669,80.7498,2025-08-22,2025,8,14260.0,67.83,8.2,3.300000e+09,8.14


### 4. Save Cleaned Data

In [241]:
# save daily data
data.to_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/Full_Ruble_Daily_Data2.csv", index=False)

In [ ]:
# save monthly data
monthly_data = data.resample('M').agg(lambda x: x.mean() if pd.api.types.is_numeric_dtype(x) else x.iloc[-1]).reset_index()
monthly_data.to_csv("/Users/zlatavorobeva/Documents/GitHub/Ruble_Volatility_Prediction2/data/Full_Ruble_Monthly_Data2.csv",index=False)
monthly_data.tail()

,date,currency_rate,year,month,GDP_per_capita,Brent_price,GDP_deflator,FD_investment,Inflation
393,2025-04-30,83.243645,2025.0,4.0,14260.0,63.37,8.2,3.300000e+09,8.14
394,2025-05-31,80.237650,2025.0,5.0,14260.0,64.32,8.2,3.300000e+09,8.14
395,2025-06-30,78.677635,2025.0,6.0,14260.0,68.15,8.2,3.300000e+09,8.14
396,2025-07-31,78.850139,2025.0,7.0,14260.0,73.43,8.2,3.300000e+09,8.14
397,2025-08-31,80.013656,2025.0,8.0,14260.0,67.83,8.2,3.300000e+09,8.14


### Summary:

This notebook prepared and cleaned historical data for RUB exchange rates and major economic indicators (GDP per capita, FDI, inflation, oil prices). 

Next steps:

- Explore trends, correlations, and structural breaks  
- Visualize RUB vs macroeconomic variables  
